# Desafio 5

Neste desafio, vamos praticar sobre redução de dimensionalidade com PCA e seleção de variáveis com RFE. Utilizaremos o _data set_ [Fifa 2019](https://www.kaggle.com/karangadiya/fifa19), contendo originalmente 89 variáveis de mais de 18 mil jogadores do _game_ FIFA 2019.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [57]:
from math import sqrt

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sct
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats as st
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

from loguru import logger

In [3]:
# Algumas configurações para o matplotlib.
# %matplotlib inline

from IPython.core.pylabtools import figsize


figsize(12, 8)

sns.set()

In [6]:
fifa = pd.read_csv("data.csv")

In [7]:
columns_to_drop = ["Unnamed: 0", "ID", "Name", "Photo", "Nationality", "Flag",
                   "Club", "Club Logo", "Value", "Wage", "Special", "Preferred Foot",
                   "International Reputation", "Weak Foot", "Skill Moves", "Work Rate",
                   "Body Type", "Real Face", "Position", "Jersey Number", "Joined",
                   "Loaned From", "Contract Valid Until", "Height", "Weight", "LS",
                   "ST", "RS", "LW", "LF", "CF", "RF", "RW", "LAM", "CAM", "RAM", "LM",
                   "LCM", "CM", "RCM", "RM", "LWB", "LDM", "CDM", "RDM", "RWB", "LB", "LCB",
                   "CB", "RCB", "RB", "Release Clause"
]

try:
    fifa.drop(columns_to_drop, axis=1, inplace=True)
except KeyError:
    logger.warning(f"Columns already dropped")

## Inicia sua análise a partir daqui

In [8]:
# Sua análise começa aqui.
fifa.head()

,Age,Overall,Potential,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,...,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes
0,31,94,94,84.0,95.0,70.0,90.0,86.0,97.0,93.0,...,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0
1,33,94,94,84.0,94.0,89.0,81.0,87.0,88.0,81.0,...,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,26,92,93,79.0,87.0,62.0,84.0,84.0,96.0,88.0,...,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0
3,27,91,93,17.0,13.0,21.0,50.0,13.0,18.0,21.0,...,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0
4,27,91,92,93.0,82.0,55.0,92.0,82.0,86.0,85.0,...,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0


In [9]:
fifa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 37 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Age              18207 non-null  int64  
 1   Overall          18207 non-null  int64  
 2   Potential        18207 non-null  int64  
 3   Crossing         18159 non-null  float64
 4   Finishing        18159 non-null  float64
 5   HeadingAccuracy  18159 non-null  float64
 6   ShortPassing     18159 non-null  float64
 7   Volleys          18159 non-null  float64
 8   Dribbling        18159 non-null  float64
 9   Curve            18159 non-null  float64
 10  FKAccuracy       18159 non-null  float64
 11  LongPassing      18159 non-null  float64
 12  BallControl      18159 non-null  float64
 13  Acceleration     18159 non-null  float64
 14  SprintSpeed      18159 non-null  float64
 15  Agility          18159 non-null  float64
 16  Reactions        18159 non-null  float64
 17  Balance     

In [12]:
fifa = fifa.dropna()

## Questão 1

Qual fração da variância consegue ser explicada pelo primeiro componente principal de `fifa`? Responda como um único float (entre 0 e 1) arredondado para três casas decimais.

In [39]:
def q1():
    pca = PCA()
    pca.fit(fifa)
    print(pca.components_)
    return float(round(pca.explained_variance_ratio_[0],3))
q1()

[[-6.16388751e-03 -3.70623864e-02 -2.27401748e-02 ...  1.91129889e-01
   1.97401130e-01  2.08994083e-01]
 [ 8.87203494e-03  1.58367355e-04 -7.78142440e-03 ... -5.14942445e-02
  -5.14745248e-02 -5.52340572e-02]
 [ 9.84206291e-02  2.04339855e-01  9.50820413e-02 ...  2.93758717e-01
   3.07480231e-01  3.20998523e-01]
 ...
 [ 1.36565256e-03  7.18377075e-03 -7.50009269e-04 ... -4.84151364e-02
  -4.47127432e-02  6.56387319e-01]
 [ 5.53542477e-01  6.30337955e-01  2.43391671e-01 ...  1.16015772e-02
  -1.80358372e-01 -1.53714829e-02]
 [ 5.64377667e-01 -6.46636967e-01  5.04199457e-01 ...  7.95845835e-03
  -1.18786610e-02  3.10873931e-02]]
[5.65280555e-01 1.81025219e-01 5.95283391e-02 4.34269284e-02
 2.63532326e-02 1.20707695e-02 1.03550495e-02 8.52951131e-03
 7.87862979e-03 7.20416677e-03 7.18180297e-03 6.32893542e-03
 5.19242286e-03 4.90798293e-03 4.74209042e-03 4.62772008e-03
 4.25173246e-03 3.86288432e-03 3.76588448e-03 3.75092615e-03
 3.47815213e-03 3.26730709e-03 3.03137944e-03 2.97415845e-0

0.565

## Questão 2

Quantos componentes principais precisamos para explicar 95% da variância total? Responda como un único escalar inteiro.

In [40]:
def q2():
    pca = PCA().fit(fifa)
    var_range = np.cumsum(pca.explained_variance_ratio_)
    print(var_range)
    for i in range(len(var_range)):
        if var_range[i]>=0.95:
            return i+1
q2()

[0.56528056 0.74630577 0.80583411 0.84926104 0.87561427 0.88768504
 0.89804009 0.9065696  0.91444823 0.9216524  0.9288342  0.93516314
 0.94035556 0.94526355 0.95000564 0.95463336 0.95888509 0.96274797
 0.96651386 0.97026478 0.97374294 0.97701024 0.98004162 0.98301578
 0.98558376 0.98788729 0.99003544 0.9915968  0.99315032 0.99423696
 0.9953052  0.99636412 0.99732226 0.99821147 0.9990712  0.99980581
 1.        ]


15

## Questão 3

Qual são as coordenadas (primeiro e segundo componentes principais) do ponto `x` abaixo? O vetor abaixo já está centralizado. Cuidado para __não__ centralizar o vetor novamente (por exemplo, invocando `PCA.transform()` nele). Responda como uma tupla de float arredondados para três casas decimais.

In [48]:
x = [0.87747123,  -1.24990363,  -1.3191255, -36.7341814,
     -35.55091139, -37.29814417, -28.68671182, -30.90902583,
     -42.37100061, -32.17082438, -28.86315326, -22.71193348,
     -38.36945867, -20.61407566, -22.72696734, -25.50360703,
     2.16339005, -27.96657305, -33.46004736,  -5.08943224,
     -30.21994603,   3.68803348, -36.10997302, -30.86899058,
     -22.69827634, -37.95847789, -22.40090313, -30.54859849,
     -26.64827358, -19.28162344, -34.69783578, -34.6614351,
     48.38377664,  47.60840355,  45.76793876,  44.61110193,
     49.28911284
]

In [51]:
def q3():
    pca = PCA(2).fit(fifa)
    return tuple(np.dot(pca.components_, x).round(3))
q3()

(186.556, -6.592)

## Questão 4

Realiza RFE com estimador de regressão linear para selecionar cinco variáveis, eliminando uma a uma. Quais são as variáveis selecionadas? Responda como uma lista de nomes de variáveis.

In [59]:
def q4():
    X = fifa.drop(['Overall'], axis=1)
    y = fifa.Overall
    reg = LinearRegression()
    rfe = RFE(reg, n_features_to_select=5, step=1)
    rfe.fit(X,y)
    return list(X.columns[rfe.support_])
q4()

['Age', 'Potential', 'BallControl', 'Reactions', 'GKReflexes']